## Test with new datasets to detect the website with trained model

In [1]:
from joblib import load
import ipaddress
import socket
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from nfstream import NFPlugin, NFStreamer
from nfstream import NFStreamer

### Loading model

In [2]:
model = load('D:\\Rayka_Company\\Azar\\Work\\ML-NFstream\\Model\\Second-version-trained-model.joblib')

### working with pcap files and convert them to CSV

In [10]:
data = NFStreamer(source='D:\\Rayka_Company\\Azar\\Work\\ML-NFstream\\Kaggle-Datasets-PCAP\\Wireshark-test\\Test1.pcapng')
# for flow in data:
#    print(flow)
DataFrame = data.to_pandas(columns_to_anonymize=[])
DataFrame.to_csv('D:\\Rayka_Company\\Azar\\Work\\ML-NFstream\\Kaggle-Datasets-PCAP\\Wireshark-test\\Test1_edit.csv', index=False)


### Loading dataset with pandas

In [11]:
DataFrame = pd.read_csv("D:\\Rayka_Company\\Azar\\Work\\ML-NFstream\\Kaggle-Datasets-PCAP\\Wireshark-test\\Test1_edit.csv")
DataFrame

,id,expiration_id,src_ip,src_mac,src_oui,src_port,dst_ip,dst_mac,dst_oui,dst_port,...,dst2src_bytes,application_name,application_category_name,application_is_guessed,application_confidence,requested_server_name,client_fingerprint,server_fingerprint,user_agent,content_type
0,0,0,10.0.103.177,28:c2:dd:e2:46:d0,28:c2:dd,53124,239.255.255.250,01:00:5e:7f:ff:fa,01:00:5e,1900,...,0,SSDP,System,0,6,239.255.255.250:1900,NaN,NaN,NaN,NaN
1,1,0,195.244.31.10,00:0c:42:d2:f6:14,00:0c:42,443,10.0.103.147,a0:29:42:94:c1:5e,a0:29:42,59944,...,108,TLS,Web,0,6,NaN,NaN,NaN,NaN,NaN
2,2,0,10.0.103.147,a0:29:42:94:c1:5e,a0:29:42,62383,10.0.202.1,00:0c:42:d2:f6:14,00:0c:42,53,...,99,DNS.Google,Network,0,6,www.google.com,NaN,NaN,NaN,NaN
3,3,0,10.0.103.147,a0:29:42:94:c1:5e,a0:29:42,49664,10.0.202.1,00:0c:42:d2:f6:14,00:0c:42,53,...,749,DNS.GoogleServices,Network,0,6,optimizationguide-pa.googleapis.com,NaN,NaN,NaN,NaN
4,4,0,10.0.103.147,a0:29:42:94:c1:5e,a0:29:42,62572,10.0.202.1,00:0c:42:d2:f6:14,00:0c:42,53,...,91,DNS.Google,Network,0,6,ssl.gstatic.com,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,357,0,10.0.103.147,a0:29:42:94:c1:5e,a0:29:42,50816,51.158.234.221,00:0c:42:d2:f6:14,00:0c:42,10799,...,9739,TLS,Web,0,6,heat-nl.tlsext.com,2658ef60ccb83e1b371ca2fd707a7bc5,f4febc55ea12b31ae17cfb7e614afda8,NaN,NaN
358,358,0,10.0.103.147,a0:29:42:94:c1:5e,a0:29:42,50823,51.158.234.221,00:0c:42:d2:f6:14,00:0c:42,10799,...,400447,TLS,Web,0,6,heat-nl.tlsext.com,b44e3777cdcaeca698efee1413dada5c,f4febc55ea12b31ae17cfb7e614afda8,NaN,NaN
359,359,0,10.0.103.147,a0:29:42:94:c1:5e,a0:29:42,50830,51.158.234.221,00:0c:42:d2:f6:14,00:0c:42,10799,...,12643,TLS,Web,0,6,heat-nl.tlsext.com,b9ccdda2f723fcb4939da8bf483140af,f4febc55ea12b31ae17cfb7e614afda8,NaN,NaN
360,360,0,10.0.103.147,a0:29:42:94:c1:5e,a0:29:42,49669,46.38.250.98,00:0c:42:d2:f6:14,00:0c:42,58266,...,2278778,WireGuard,VPN,0,6,NaN,NaN,NaN,NaN,NaN


### Preprocessing on dataset

In [12]:
def ip_to_int(ip):
    try:
        ip_obj = ipaddress.ip_address(ip)
        if isinstance(ip_obj, ipaddress.IPv4Address):
            ip_int = int.from_bytes(socket.inet_aton(ip), 'big')
            return np.uint32(ip_int)
        elif isinstance(ip_obj, ipaddress.IPv6Address):
            return int(ip_obj)
    except ValueError as e:
        print(f"Invalid IP address {ip}: {e}")
        return np.nan

# Assuming df_Final is your DataFrame
DataFrame["src_ip"] = DataFrame['src_ip'].apply(ip_to_int)
DataFrame["dst_ip"] = DataFrame['dst_ip'].apply(ip_to_int)

In [13]:
DataFrame = DataFrame[['src_ip', 'dst_ip', 'application_category_name','application_name']]
# df = df.dropna()

### Prediction

In [14]:
X_new_test = DataFrame.drop(['application_name', 'application_category_name'], axis=1)
# Predict with the loaded model
predictions_new_dataset1 = model.predict(X_new_test)
predictions_new_dataset1
# print(len(predictions_new_dataset1))
# If you want to add predictions back to your DataFrame

array([['TLS', 'Web'],
       ['TLS', 'Web'],
       ['TLS.Steam', 'Game'],
       ['TLS.Steam', 'Game'],
       ['TLS.Steam', 'Game'],
       ['TLS.Steam', 'Game'],
       ['TLS.Steam', 'Game'],
       ['SSDP', 'System'],
       ['TLS.Steam', 'Game'],
       ['TLS.Steam', 'Game'],
       ['TLS.Steam', 'Game'],
       ['TLS.Steam', 'Game'],
       ['TLS.Steam', 'Game'],
       ['TLS.Steam', 'Game'],
       ['TLS', 'Web'],
       ['TLS.Steam', 'Game'],
       ['TLS.Steam', 'Game'],
       ['TLS.Steam', 'Game'],
       ['TLS.Steam', 'Game'],
       ['TLS.Steam', 'Game'],
       ['TLS.Steam', 'Game'],
       ['SSDP', 'System'],
       ['TLS', 'Web'],
       ['TLS.Steam', 'Game'],
       ['TLS.Steam', 'Game'],
       ['TLS.Steam', 'Game'],
       ['TLS', 'Web'],
       ['TLS.Steam', 'Game'],
       ['TLS.Steam', 'Game'],
       ['SSDP', 'System'],
       ['TLS.Steam', 'Game'],
       ['TLS.Steam', 'Game'],
       ['TLS.Steam', 'Game'],
       ['TLS.Steam', 'Game'],
       ['SSDP', 'System'

In [19]:
DataFrame['predicted_application_name'] = predictions_new_dataset1[:, 0]
DataFrame['predicted_website_category'] = predictions_new_dataset1[:, 1]

C:\Users\Arina\AppData\Local\Temp\ipykernel_28176\200456309.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DataFrame['predicted_application_name'] = predictions_new_dataset1[:, 0]
C:\Users\Arina\AppData\Local\Temp\ipykernel_28176\200456309.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DataFrame['predicted_website_category'] = predictions_new_dataset1[:, 1]


In [23]:
predictions_df = pd.DataFrame(predictions_new_dataset1, columns=['predicted_value_1', 'predicted_value_2'])  # Adjust column names as needed

y_test = DataFrame[['application_category_name','application_name']]
combined_results = pd.concat([predictions_df, y_test], axis=1)
combined_results.to_csv('prediction_results.csv', index=False)


In [25]:
importances = model.feature_importances_
for i in range(len(importances)):
    print(f"Feature {X_new_test.columns[i]}: {importances[i]}")

Feature src_ip: 0.6095300165091969
Feature dst_ip: 0.39046998349080303


### Get result of predicting in csv file

In [15]:
predictions_df_new = pd.DataFrame(predictions_new_dataset1, columns=['predicted_application_name', 'predicted_application_category_name'])
result_df = pd.concat([X_new_test.reset_index(drop=True), predictions_df_new], axis=1)
result_df.to_csv('D:\\Rayka_Company\\Azar\Work\\ML-NFstream\\Kaggle-Datasets-PCAP\\Wireshark-test\\model_predictions_test1_edit.csv', index=False)